In [3]:
import numpy as np
import pandas as pd
import os
import langchain
import lxml
from lxml import etree
from bs4 import BeautifulSoup
import sqlite3
import xml.etree.ElementTree as ET
from gensim.utils import simple_preprocess
from gensim.corpora.dictionary import Dictionary
from gensim.models import TfidfModel
from gensim import similarities

In [4]:
#conda update -n base conda

In [5]:
#conda update --all

In [6]:
#conda install conda=23.11.0

In [7]:
#conda install -c conda-forge langchain

In [8]:
#conda install langchain -c conda-forge

In [9]:
#conda install -c conda-forge lxml

In [10]:
#conda install -c conda-forge gensim

In [11]:
#conda install -c conda-forge os

In [12]:
#conda install -c conda-forge elasticsearch

In [13]:
data = 'data/rawrrr/'

In [14]:
input_files = os.listdir(data)

In [15]:
sample_size=100
try:
    sample_input_files = input_files[0:100]
except ValueError:
    sample_input_files = input_files

In [16]:
sample_input_files

['168988177.xml',
 '168990737.xml',
 '168985651.xml',
 '169003396.xml',
 '169011826.xml',
 '168968697.xml',
 '168957808.xml',
 '168962247.xml',
 '168966961.xml',
 '168956322.xml',
 '168968911.xml',
 '168940782.xml',
 '168982387.xml',
 '168980197.xml',
 '169010136.xml',
 '168966546.xml',
 '168986645.xml',
 '168997709.xml',
 '168997394.xml',
 '168986498.xml',
 '168963421.xml',
 '168975566.xml',
 '168989108.xml',
 '168966488.xml',
 '168980155.xml',
 '168986616.xml',
 '168960236.xml',
 '168974541.xml',
 '168976830.xml',
 '168956160.xml',
 '168988678.xml',
 '168969060.xml',
 '168998117.xml',
 '168979215.xml',
 '168975532.xml',
 '168966537.xml',
 '168963798.xml',
 '168978710.xml',
 '168967192.xml',
 '168971311.xml',
 '168956699.xml',
 '168969945.xml',
 '168982846.xml',
 '168952935.xml',
 '168971534.xml',
 '169005926.xml',
 '168984845.xml',
 '168948373.xml',
 '169010632.xml',
 '168961651.xml',
 '168997757.xml',
 '168983787.xml',
 '169006374.xml',
 '168962839.xml',
 '168964020.xml',
 '16900997

In [17]:
# Function to strip html tags from text portion
def strip_html_tags(text):
    stripped = BeautifulSoup(text).get_text().replace('\n', ' ').replace('\\', '').strip()
    return stripped

In [18]:
# Retrieve metadata from XML document
def getxmlcontent(corpus, file, strip_html=True):
    try:
        tree = etree.parse(corpus + file)
        root = tree.getroot()

        if root.find('.//GOID') is not None:
            goid = root.find('.//GOID').text
        else:
            goid = None

        if root.find('.//Title') is not None:
            title = root.find('.//Title').text
        else:
            title = None

        if root.find('.//NumericDate') is not None:
            date = root.find('.//NumericDate').text
        else:
            date = None
            
        if root.find('.//PublisherName') is not None:
            publisher = root.find('.//PublisherName').text
        else:
            publisher = None

        if root.find('.//FullText') is not None:
            text = root.find('.//FullText').text

        elif root.find('.//HiddenText') is not None:
            text = root.find('.//HiddenText').text

        elif root.find('.//Text') is not None:
            text = root.find('.//Text').text

        else:
            text = None

        # Strip html from text portion
        if text is not None and strip_html == True:
            text = strip_html_tags(text)
    
    except Exception as e:
        print(f"Error while parsing file {file}: {e}")
    
    return goid, title, date, publisher, text

In [19]:
# Create lists to store article IDs, titles, dates, and text
goid_list = []
title_list = []
date_list = []
text_list = []

# Parse files and add data to lists
for file in sample_input_files:
    
    # Retrieve the metadata
    goid, title, date, publisher, text = getxmlcontent(data, file, strip_html=True)
    
    # Store metadata to lists
    goid_list.append(goid)
    title_list.append(title)
    date_list.append(date)
    text_list.append(text)

In [20]:
df = pd.DataFrame({'GOID': goid_list,'Title': title_list, 'Date': date_list, 'Text': text_list })

In [21]:
df

,GOID,Title,Date,Text
0,168988177,Article 1 -- No Title,1970-01-12,A Mliwauikee-based legal research or- ganizati...
1,168990737,Display Ad 38 -- No Title,1970-01-22,Where There's A Niliman's . . . There Are Lowe...
2,168985651,Weekly Word Game,1970-01-10,We/dy VWord 7ame A Brain Teaser for Puzzle F...
3,169003396,"FREUD LOVED WIFE'S SISTER, DOCTOR SAYS",1970-01-06,"FREUD LOVED WIFE S SISTER, DOCTOR SAYS Psy..."
4,169011826,Cake Shells for Next Christmas,1970-01-06,Cake Shells for Next Christmas BY MARY MEADE...
...,...,...,...,...
95,168945157,Judge Ethics Group Makes Ist Rulings,1970-01-26,"Judge Ethics Group Makes Ist Rulings ""Washin..."
96,168970575,Stock Quote 10 -- No Title,1970-01-23,Board of Trade Pricesi 2 22. 19701 GrAlns...
97,168975797,Stricken Super-tanker Makes Liberian Port,1970-01-02,Stricken Super-tanker I Makes Liberian Port ...
98,168960119,HATTIE WEIL'S ESTATE IS PUT AT 1.5 MILLION,1970-01-27,HATTIE WEIL S I ESTATE IS PUT AT 1.5 MILLION ...


In [22]:
conn = sqlite3.connect('test_data.db')
cursor = conn.cursor()

In [23]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS your_table (
        goid INTEGER PRIMARY KEY,
        title TEXT,
        date TEXT,
        publication TEXT,
        text TEXT
    )
''')
conn.commit()

In [24]:
def insert_data_from_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Example: Extracting values with handling for missing elements or attributes
    goid = root.find('.//GOID').text if root.find('.//GOID') is not None else None
    title = root.find('.//Title').text if root.find('.//Title') is not None else None
    date = root.find('.//NumericDate').text if root.find('.//NumericDate') is not None else None
    publication = root.find('.//PublishaerName').text if root.find('.//PublisherName') is not None else None
#     text = root.find('.//FullText').text if root.find('.//FullText') is not None else None
    if root.find('.//FullText') is not None:
        text = root.find('.//FullText').text

    elif root.find('.//HiddenText') is not None:
        text = root.find('.//HiddenText').text

    elif root.find('.//Text') is not None:
        text = root.find('.//Text').text

    else:
        text = None

    if text is not None:
        text = strip_html_tags(text)
    

    # Use the extracted values as needed (e.g., insert into a database)
#     print("Text:", text)
    # Extract other fields as needed

    # Insert data into SQLite
    cursor.execute('''
        INSERT INTO your_table (goid, title, date, publication, text)
        VALUES (?, ?, ?, ?, ?)
    ''', (goid, title, date, publication, text))
    # Adjust the query and parameters based on your table schema

# Specify the directory containing your XML files
xml_directory = data

# Iterate through XML files in the directory and insert data into SQLite
for filename in os.listdir(xml_directory):
    if filename.endswith('.xml'):
        xml_file_path = os.path.join(xml_directory, filename)
        insert_data_from_xml(xml_file_path)

# Commit changes and close the database connection
conn.commit()
conn.close()

In [27]:
conn = sqlite3.connect('test_data.db')
cursor = conn.cursor()

# Fetch data from the SQLite database
cursor.execute("SELECT goid, text FROM your_table")
rows = cursor.fetchall()

# Example: Load your data
# Replace this with your actual method of loading articles
documents = [row[1] for row in rows if row[1] is not None]

# Preprocess documents
processed_docs = [simple_preprocess(doc) for doc in documents]

# Create a dictionary representation of the documents
dictionary = Dictionary(processed_docs)

# Create a corpus from the dictionary and processed documents
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Create a TF-IDF model from the corpus
tfidf = TfidfModel(corpus)

# Create an index for similarity queries
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=len(dictionary))

def search(query):
    query_bow = dictionary.doc2bow(simple_preprocess(query))
    query_tfidf = tfidf[query_bow]
    similarities = index[query_tfidf]
    return sorted(enumerate(similarities), key=lambda item: -item[1])

# Example search
search_results = search("nixon")

# Process and use search results as needed
for result in search_results:
    article_id, similarity_score = rows[result[0]][0], result[1]
    print(f"Article ID: {article_id}, Similarity Score: {similarity_score}")

# Close connections
cursor.close()
conn.close()

Article ID: 169002696, Similarity Score: 0.44861161708831787
Article ID: 168996998, Similarity Score: 0.4470128118991852
Article ID: 168980434, Similarity Score: 0.4306938350200653
Article ID: 168969843, Similarity Score: 0.42992836236953735
Article ID: 168955157, Similarity Score: 0.4285052716732025
Article ID: 168979595, Similarity Score: 0.3809170722961426
Article ID: 168972728, Similarity Score: 0.37974753975868225
Article ID: 168991373, Similarity Score: 0.367678701877594
Article ID: 168969010, Similarity Score: 0.3646695911884308
Article ID: 168958271, Similarity Score: 0.36139604449272156
Article ID: 168964774, Similarity Score: 0.3523799479007721
Article ID: 168986034, Similarity Score: 0.3298660218715668
Article ID: 168983562, Similarity Score: 0.3276667296886444
Article ID: 168955464, Similarity Score: 0.32137706875801086
Article ID: 168962286, Similarity Score: 0.32137706875801086
Article ID: 168961043, Similarity Score: 0.320136159658432
Article ID: 168940138, Similarity Sc

In [28]:
# Process and use search results as needed
article = []
for result in search_results[0:10]:
    article.append(rows[result[0]][0])

In [29]:
article

[169002696,
 168996998,
 168980434,
 168969843,
 168955157,
 168979595,
 168972728,
 168991373,
 168969010,
 168958271]

In [30]:
conn = sqlite3.connect('test_data.db')
cursor = conn.cursor()

In [31]:
query = 'SELECT * FROM your_table WHERE GOID IN ({})'.format(','.join('?' * len(article))) 
cursor.execute(query, article)

In [32]:
rows = cursor.fetchall()

In [33]:
conn.close()